In [6]:
import os
import langchain_chroma
import openai
from openai import OpenAI
import chromadb
import fitz  # PDF에서 텍스트 추출
from dotenv import load_dotenv
from fastapi import FastAPI, UploadFile, File
from sqlalchemy import create_engine, text
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import psycopg2
import torch
from sklearn.metrics.pairwise import cosine_similarity
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from getpass import getpass
from langchain.vectorstores import ElasticVectorSearch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.storage import InMemoryStore
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [4]:
# .env 파일 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
df = pd.read_csv("../data_backup/rec_data.csv")

In [13]:
# 이력서 불러오기
path = "./data/빅데이터AI_이력서.pdf"
doc = fitz.open(path)
for page in doc:
    resume_text = page.get_text()
    print(resume_text)

📄 이력서 (Resume)
[기본 정보]
이름: 홍길동
연락처: 010-1234-5678
이메일: honggildong.ai@gmail.com
주소: 서울특별시 강남구 테헤란로 123
[학력]
고려대학교 컴퓨터학과 졸업 (2018.03 ~ 2024.02)
GPA: 3.85 / 4.5
관련 과목: 머신러닝, 데이터마이닝, 통계학, 빅데이터처리, 딥러닝 이론과 실습
[기술 스택]
Programming: Python, SQL, R
Frameworks/Libraries: Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy
Tools: Jupyter, Git, Docker, Tableau
DBMS: MySQL, MongoDB, Hadoop(HDFS), Spark
Cloud: Google Colab, AWS EC2 & S3 (기초 수준)
[프로젝트 경험]
1. 신문 기사 기반 감성 분석 모델 개발 (2023.03 ~ 2023.06)
자연어처리(NLP) 기반 감성 분류 모델 개발
KoNLPy와 Scikit-learn을 이용한 전처리 및 모델 학습
정확도 86% 달성
2. 머신러닝 기반 개인 맞춤형 영화 추천 시스템 (2023.09 ~ 2023.12)
📄 이력서 (Resume)
1

Content-based Filtering 및 Collaborative Filtering 기법 적용
Streamlit으로 웹 인터페이스 구현
kaggle 데이터셋 기반, Precision@10: 0.73
[자격증]
ADsP (데이터분석 준전문가) – 2023.08
SQLD (SQL 개발자) – 2024.01
📄 이력서 (Resume)
2



In [7]:
index_name = "jd_index"
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "elastic7276"),
    verify_certs=False
)

/root/miniconda3/envs/apps/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:403: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [8]:
def index_documents_to_es(index_name, documents):
    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name)

    actions = [
        {
            "_index": index_name,
            "_id": i,
            "_source": {
                "content": doc.page_content,
                **doc.metadata
            }
        }
        for i, doc in enumerate(documents)
    ]

    bulk(es, actions)

In [9]:
class ElasticsearchRetriever:
    def __init__(self, es, index_name, k=5):
        self.es = es
        self.index_name = index_name
        self.k = k

    def get_relevant_documents(self, query):
        response = self.es.search(
            index=self.index_name,
            query={
                "match": {
                    "content": {
                        "query": query,
                        "operator": "and"
                    }
                }
            },
            size=self.k
        )
        hits = response["hits"]["hits"]
        return [
            Document(
                page_content=hit["_source"]["content"],
                metadata={k: v for k, v in hit["_source"].items() if k != "content"}
            )
            for hit in hits
        ]

In [37]:
class HybridRetriever:
    def __init__(self, es_retriever, vector_retriever, alpha=0.5):
        self.es_retriever = es_retriever
        self.vector_retriever = vector_retriever
        self.alpha = alpha

    def get_relevant_documents(self, query):
        es_docs = self.es_retriever.get_relevant_documents(query)
        vec_docs = self.vector_retriever.get_relevant_documents(query)

        combined = es_docs[:int((1 - self.alpha) * 10)] + vec_docs[:int(self.alpha * 10)]

        # 중복 제거
        seen = set()
        final = []
        for doc in combined:
            key = doc.metadata.get("rec_idx")
            if key not in seen:
                final.append(doc)
                seen.add(key)
        return final


In [38]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

vectordb = Chroma(
    persist_directory="./chroma_data",
    embedding_function=embedding_model,
    collection_name="chroma_test"
)

vector_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [39]:
es_retriever = ElasticsearchRetriever(es, index_name, k=10)

In [40]:
hybrid_retriever = HybridRetriever(es_retriever, vector_retriever, alpha=0.3)

In [41]:
query = resume_text
docs = hybrid_retriever.get_relevant_documents(query)

/root/miniconda3/envs/apps/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
print(context)

4

9

9

0

2

7

3

9




[

서

울

]

 

블

리

비

의

원

 

상

담

실

장

 

신

규

/

추

가

 

모

집










사

람

인

 




 




 




 




본

문

 

바

로

가

기




2

0

2

5

년

 

상

반

기

 

공

채

 

시

작

!




2

0

2

5

년

 

상

반

기

 

공

채

 

시

작

!




로

그

인







회

원

가

입




기

업

서

비

스




로

그

인




회

원

가

입







기

업

홈




공

고

 

등

록




지

원

자

 

관

리




인

재

풀




스

마

트

 

리

크

루

터




인

적

성

 

·

 

평

가

도

구




H

R

매

거

진




채

용

상

품




사

람

인

 

비

즈

니

스




전

체

메

뉴




채

용

정

보




지

역

별




직

업

별




역

세

권

별




H

O

T

1

0

0




헤

드

헌

팅




채

용

관




큐

레

이

션




파

견

대

행




외

국

인

 

채

용




취

업

축

하

금




상

반

기

 

공

채

신

입

·

인

턴

 




신

입

·

인

턴

 

홈




실

시

간

 

공

고




채

용

달

력




신

입

연

봉




자

소

서




인

적

성

검

사




인

적

성

검

사




공

기

업

 

모

의

고

사




기

업

·

연

봉




기

업

리

뷰




연

봉

정

보




면

접

후

기




기

업

큐

레

이

션




커

리

어

성

장

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

context = "\n\n".join(f"\n{doc.page_content.strip()}")

template = f"""
당신은 인재매칭 AI 어시스턴트입니다. 사용자 이력서에 기반하여 채용공고를 추천해주세요.

- 반드시 이력서에 기반할 것.
- 채용공고 상세는 반드시 모두 출력할 것.
- 출력 시 채용공고의 양식을 사용할 것.
- 채용공고 추천 이유를 한줄로 설명할 것.         
- 같은 공고 번호가 중복될 경우 단 1개만 추천할 것.                  
- 채용공고 전체 내용을 기반하여 분석할 것. 제목만 보고 판단하지 말 것.
- page_content의 전체 텍스트를 기준으로 판단할 것.
- [채용공고]에 존재하지 않는 정보는 출력하지 말 것.

### 출력 형식 예시
1.
- 직무명 : 
- 공고 번호 :                         
- 산업 분야 : 
- 필요 기술 : 
- 채용공고 상세 :
- 추천 사유 :

이 정보를 바탕으로 사용자 정보에 가장 적합한 채용 공고 5개를 추천해주세요.
출력 형식과 동일한 형식으로 출력해주세요.                                          

[채용공고]:
{context}

[사용자 이력서]:
{query}
"""

prompt = PromptTemplate.from_template(template)

# LLM 세팅
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)



# hybrid_retriever를 Runnable로 래핑
context_chain = RunnableLambda(hybrid_retriever.get_relevant_documents) | (lambda docs: "\n\n".join(
    f"{doc.metadata.get('rec_idx')}\n{doc.page_content.strip()}" for doc in docs
))

rag_chain = (
    {
        "context": context_chain,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [59]:
response = rag_chain.invoke(resume_text)
print(response)

/root/miniconda3/envs/apps/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


사용자 이력서를 기반으로 적합한 채용 공고를 추천해 드리겠습니다. 사용자의 데이터 분석 및 개발 관련 경험과 자격증을 고려하여 다음과 같은 채용 공고를 추천합니다.

1.
- 직무명 : 데이터 분석가
- 공고 번호 : 2025-001
- 산업 분야 : IT 및 데이터 분석
- 필요 기술 : 데이터 분석, SQL, Python, 머신러닝
- 채용공고 상세 : 데이터 분석 및 모델링, 대규모 데이터셋 처리, 비즈니스 인사이트 도출
- 추천 사유 : 사용자의 데이터 분석 경험과 ADsP 자격증이 이 직무와 잘 맞습니다.

2.
- 직무명 : 머신러닝 엔지니어
- 공고 번호 : 2025-002
- 산업 분야 : 인공지능 및 머신러닝
- 필요 기술 : 머신러닝, Python, TensorFlow, 데이터 전처리
- 채용공고 상세 : 머신러닝 모델 개발 및 최적화, 데이터 파이프라인 구축
- 추천 사유 : 사용자의 머신러닝 기법 적용 경험이 이 직무에 적합합니다.

3.
- 직무명 : 데이터 사이언티스트
- 공고 번호 : 2025-003
- 산업 분야 : 금융 데이터 분석
- 필요 기술 : 데이터 분석, R, Python, 통계 분석
- 채용공고 상세 : 금융 데이터 분석 및 예측 모델 개발, 데이터 시각화
- 추천 사유 : 사용자의 데이터 분석 능력과 SQLD 자격증이 이 직무에 적합합니다.

4.
- 직무명 : SQL 개발자
- 공고 번호 : 2025-004
- 산업 분야 : 데이터베이스 관리
- 필요 기술 : SQL, 데이터베이스 설계, 데이터 마이그레이션
- 채용공고 상세 : 데이터베이스 설계 및 최적화, 데이터 마이그레이션 프로젝트 수행
- 추천 사유 : 사용자의 SQLD 자격증과 관련 경험이 이 직무에 적합합니다.

5.
- 직무명 : 웹 애플리케이션 개발자
- 공고 번호 : 2025-005
- 산업 분야 : 웹 개발
- 필요 기술 : Python, Django, 웹 인터페이스 개발
- 채용공고 상세 : 웹 애플리케이션 개발 및 유지보수, 사용자 인터페이스 최